In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import create_engine,inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session


In [3]:
Vancouver_csv_file = "../Resources/crime_csv_all_years.csv"
Vancouver_csv_df = pd.read_csv(Vancouver_csv_file)
Vancouver_csv_df.head()

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
0,Theft from Vehicle,2003,4,29,14.0,0.0,23XX W 7TH AVE,Kitsilano,488378.25,5457008.26
1,Break and Enter Commercial,2003,11,1,5.0,0.0,23XX E HASTINGS ST,Grandview-Woodland,495820.60,5458719.06
2,Theft of Vehicle,2003,5,18,17.0,0.0,10XX E BROADWAY AVE,Mount Pleasant,493944.67,5456629.46
3,Break and Enter Residential/Other,2003,4,1,10.0,0.0,54XX WALES ST,Renfrew-Collingwood,496232.24,5453579.56
4,Theft of Vehicle,2003,12,11,9.0,0.0,26XX BLANCA ST,West Point Grey,484344.26,5456805.39


In [5]:
new_Vancouver_csv_df = Vancouver_csv_df[['TYPE', 'YEAR', 'MONTH', 'DAY', 'X', 'Y']].copy()
new_Vancouver_csv_df.head()

,TYPE,YEAR,MONTH,DAY,X,Y
0,Theft from Vehicle,2003,4,29,488378.25,5457008.26
1,Break and Enter Commercial,2003,11,1,495820.60,5458719.06
2,Theft of Vehicle,2003,5,18,493944.67,5456629.46
3,Break and Enter Residential/Other,2003,4,1,496232.24,5453579.56
4,Theft of Vehicle,2003,12,11,484344.26,5456805.39


In [6]:
renamed_Vancouver_df = new_Vancouver_csv_df.rename(columns={"TYPE":"crimestype" , "YEAR":"year", "MONTH":"month", "DAY": "day", "X": "latitude",  "Y":"longitude"})
renamed_Vancouver_df.head()

,crimestype,year,month,day,latitude,longitude
0,Theft from Vehicle,2003,4,29,488378.25,5457008.26
1,Break and Enter Commercial,2003,11,1,495820.60,5458719.06
2,Theft of Vehicle,2003,5,18,493944.67,5456629.46
3,Break and Enter Residential/Other,2003,4,1,496232.24,5453579.56
4,Theft of Vehicle,2003,12,11,484344.26,5456805.39


In [7]:
renamed_Vancouver_df = renamed_Vancouver_df[renamed_Vancouver_df['crimestype'].isin (["Theft from Vehicle","Break and Enter Commercial", "Break and Enter Residential/Other", "Offence Against a Person"])]
                                           #(["Theft from Vehicle", "Break and Enter Commercial", "Break and Enter Residential/Other", "Offence Against a Person"])
renamed_Vancouver_df.head()

,crimestype,year,month,day,latitude,longitude
0,Theft from Vehicle,2003,4,29,488378.25,5457008.26
1,Break and Enter Commercial,2003,11,1,495820.60,5458719.06
3,Break and Enter Residential/Other,2003,4,1,496232.24,5453579.56
5,Break and Enter Residential/Other,2003,12,5,489995.20,5459563.59
6,Offence Against a Person,2003,4,4,0.00,0.00


In [8]:
listt= ["Theft from Vehicle","Break and Enter Commercial", "Break and Enter Residential/Other", "Offence Against a Person"]
df_crime_type = pd.DataFrame({"type": listt})
df_crime_type

,type
0,Theft from Vehicle
1,Break and Enter Commercial
2,Break and Enter Residential/Other
3,Offence Against a Person


In [9]:
rds_connection_string = "postgres:Rav6islexus!!!@localhost:5432/Crime"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
engine.table_names()

['city',
 'registercrime',
 'crimestype',
 'census',
 'vancouver_crime',
 'Vancouver_crime']

In [11]:
# identify the tables name
tables = engine.table_names()

In [12]:
# identify the internal columns of the table
inspector = inspect(engine)

In [13]:
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [14]:
#Table object creation
City = Base.classes.city
CrimeType = Base.classes.crimestype
RegisterCrime = Base.classes.registercrime

In [15]:
query = session.query(City).filter(City.name=="Vancouver").all()
print(query)
if query == []:
    city_one = City(name='Vancouver', country="Canada", state="ON")
    session.add(city_one)
    session.commit()

In [16]:
#deleting content of the table, after that insert 
engine.execute("delete FROM registercrime")
session.commit()

In [17]:
#new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)
df_crime_type.to_sql(name='crimestype', con=engine, if_exists='append', index=False)
session.commit()

In [25]:
session.rollback()

In [27]:
for index, row in renamed_Vancouver_df.iterrows():
    id_crime = session.query(CrimeType.id).filter(CrimeType.type == row["crimestype"]).first()
    year = row["year"]
    month = row["month"]
    day = row["day"]
    latitude = row["latitude"]
    longitude = row["longitude"]
    id_city = session.query(City.id).filter(City.name=='Vancouver').first()
    session.add(RegisterCrime(id_crime = id_crime, id_city= id_city, year=year, month=month, day=day, latitude=latitude, longitude=longitude))
    session.commit()